#### **G&F Digitizing**

# Convert paper scans from Griffin & Critchfield's into georeferenced digital formats, creating accurate range maps.

------------------------------------------------------------------------------

#### **Project Code**

* Created by Jason Yuen.


### Content 

* Running the necessary functions
* Setting the file paths
* Digitizing and running the script

* Clipping feature classes to boundary

#### 1. Running the necessary functions. 

We'll need to run this everytime we load into the script in order to remove the user's manual inputting of parameters into geoprocessing. 

In [ ]:
# Functions

def project(speciesOfInterest):
    # projects 1927 to 1983
    arcpy.management.ProjectRaster(
        in_raster=workflowFolderLoc + f"georef_1927\\{speciesOfInterest}.tif",
        out_raster=workflowFolderLoc + f"georef_1983\\{speciesOfInterest}83.tif",
        out_coor_system='PROJCS["NAD_1983_2011_California_Teale_Albers",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
        resampling_type="NEAREST",
        cell_size="213.933429069565 214.320035128142",
        geographic_transform="'NAD_1927_To_WGS_1984_79_CONUS + WGS_1984_(ITRF08)_To_NAD_1983_2011'",
        Registration_Point=None,
        in_coor_system='PROJCS["NAD_1927_California_Teale_Albers",GEOGCS["GCS_North_American_1927",DATUM["D_North_American_1927",SPHEROID["Clarke_1866",6378206.4,294.9786982]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
        vertical="NO_VERTICAL"
    )

def createFC(speciesOfInterest):
    # uses an if-else statement to create feature classes for points
    if (pointResult == "yes"):
        arcpy.management.CreateFeatureclass(
            out_path=geodatabase,
            out_name=f"{speciesOfInterest}_points",
            geometry_type="POINT",
            template=None,
            has_m="DISABLED",
            has_z="DISABLED",
            spatial_reference='PROJCS["NAD_1983_2011_California_Teale_Albers",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-16909700 -8597000 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision',
            config_keyword="",
            spatial_grid_1=0,
            spatial_grid_2=0,
            spatial_grid_3=0,
            out_alias="",
            oid_type="SAME_AS_TEMPLATE"
        )
    else:
        print("No stands less than 2 miles across or of unknown size.")

    # uses an if-else statement to create feature classes for polygons
    if (polygonResult == "yes"):
        arcpy.management.CreateFeatureclass(
            out_path=geodatabase,
            out_name=f"{speciesOfInterest}_blob_polyline",
            geometry_type="POLYLINE",
            template=None,
            has_m="DISABLED",
            has_z="DISABLED",
            spatial_reference='PROJCS["NAD_1983_2011_California_Teale_Albers",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-16909700 -8597000 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision',
            config_keyword="",
            spatial_grid_1=0,
            spatial_grid_2=0,
            spatial_grid_3=0,
            out_alias="",
            oid_type="SAME_AS_TEMPLATE"
        )
    else:
        print("No groups of stands more than 2 miles across.")


def postDigitize(speciesOfInterest):
    if (pointResult == "yes"):
        # buffer points
        arcpy.analysis.Buffer(
            in_features = f"{geodatabase}\\{speciesOfInterest}_points",
            out_feature_class = f"{geodatabase}\\{speciesOfInterest}_buffered",
            buffer_distance_or_field="1 Miles",
            line_side="FULL",
            line_end_type="ROUND",
            dissolve_option="NONE",
            dissolve_field=None,
            method="PLANAR"
        )
        # dissolve the buffer
        arcpy.management.Dissolve(
            in_features=f"{geodatabase}\\{speciesOfInterest}_buffered",
            out_feature_class=f"{geodatabase}\\{speciesOfInterest}_dissolved_points",
            dissolve_field=None,
            statistics_fields=None,
            multi_part="SINGLE_PART",
            unsplit_lines="DISSOLVE_LINES",
            concatenation_separator=""
        )
        # add a field
        arcpy.management.AddField(
            in_table=f"{geodatabase}\\{speciesOfInterest}_dissolved_points",
            field_name="Level",
            field_type="TEXT"
        )
        # calculate field
        arcpy.management.CalculateField(
            in_table=f"{geodatabase}\\{speciesOfInterest}_dissolved_points",
            field="Level",
            expression="2",
            expression_type="PYTHON3",
            code_block="",
            field_type="TEXT",
            enforce_domains="NO_ENFORCE_DOMAINS"
        )
    else:
        print("No stands less than 2 miles across or of unknown size.")

    if (polygonResult == "yes"):
        # changes polylines too polygons
        arcpy.management.FeatureToPolygon(
            in_features=geodatabase + f"\\{speciesOfInterest}_blob_polyline",
            out_feature_class=f"{geodatabase}\\{speciesOfInterest}_level_1",
            cluster_tolerance=None,
            attributes="ATTRIBUTES",
            label_features=None
        )
        # add a field
        arcpy.management.AddField(
            in_table= f"{geodatabase}\\{speciesOfInterest}_level_1",
            field_name="Level",
            field_type="TEXT"
        )
        # calculate field
        arcpy.management.CalculateField(
            in_table=f"{geodatabase}\\{speciesOfInterest}_level_1",
            field="Level",
            expression="1",
            expression_type="PYTHON3",
            code_block="",
            field_type="TEXT",
            enforce_domains="NO_ENFORCE_DOMAINS"
        )
    else:
        print("No groups of stands more than 2 miles across.")

def postEdit(speciesOfInterest):
    arcpy.analysis.Update(
        in_features=f"{geodatabase}\\{speciesOfInterest}_dissolved_points",
        update_features=f"{geodatabase}\\{speciesOfInterest}_level_1",
        out_feature_class=f"{geodatabase}\\{speciesOfInterest}_update",
        keep_borders="BORDERS",
        cluster_tolerance=None
    )

    arcpy.conversion.ExportFeatures(
        in_features=f"{geodatabase}\\{speciesOfInterest}_update",
        out_features=f"{geodatabase}\\{speciesOfInterest}_level_2",
        where_clause="Level = '2'",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping=f'Level "Level" true true false 255 Text 0 0,First,#,{speciesOfInterest}_update,Level,0,254;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,{speciesOfInterest}_update,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,{speciesOfInterest}_update,Shape_Area,-1,-1',
        sort_field=None
    )

def postEdit2(speciesOfInterest):
    arcpy.conversion.ExportFeatures(
        in_features=f"{speciesOfInterest}_dissolved_points",
        out_features=f"{geodatabase}\\{speciesOfInterest}_level_2",
        where_clause="",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping=f'Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,{speciesOfInterest}_dissolved_points,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,{speciesOfInterest}_dissolved_points,Shape_Area,-1,-1;Level "Level" true true false 255 Text 0 0,First,#,{speciesOfInterest}_dissolved_points,Level,0,254',
        sort_field=None
    )

    arcpy.conversion.ExportFeatures(
        in_features=f"{speciesOfInterest}_dissolved_points",
        out_features=f"{geodatabase}\\{speciesOfInterest}_update",
        where_clause="",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping=f'Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,{speciesOfInterest}_dissolved_points,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,{speciesOfInterest}_dissolved_points,Shape_Area,-1,-1;Level "Level" true true false 255 Text 0 0,First,#,{speciesOfInterest}_dissolved_points,Level,0,254',
        sort_field=None
    )


def changeUpdate(speciesOfInterest):
    arcpy.management.PolygonToLine(
      in_features=f"{geodatabase}\\{speciesOfInterest}_update",
      out_feature_class=f"{geodatabase}\\{speciesOfInterest}_to_line",
      neighbor_option="IDENTIFY_NEIGHBORS"
    )

def dashLinesToPolygon(speciesOfInterest):
    arcpy.management.FeatureToPolygon(
      in_features=f"{geodatabase}\\{speciesOfInterest}_to_line",
      out_feature_class=f"{geodatabase}\\{speciesOfInterest}_to_line_polygon",
      cluster_tolerance=None,
      attributes="ATTRIBUTES",
      label_features=None
  )

def finalStep(speciesOfInterest):
    updateFile = f"{geodatabase}\\{speciesOfInterest}_update"

    with arcpy.da.UpdateCursor(updateFile, ['Level']) as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = "3"
                cursor.updateRow(row)

    arcpy.conversion.ExportFeatures(
        in_features=f"{geodatabase}\\{speciesOfInterest}_update",
        out_features=f"{geodatabase}\\{speciesOfInterest}_level_3",
        where_clause="Level = '3'",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping=f'Level "Level" true true false 255 Text 0 0,First,#,{speciesOfInterest}_update,Level,0,254;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,{speciesOfInterest}_update,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,{speciesOfInterest}_update,Shape_Area,-1,-1',
        sort_field=None
    )


In [ ]:
def executeFunction(userInput):
  # Check if any rows match the user input
  matchedRow = functionsDF[functionsDF['Function title'] == userInput]

  if not matchedRow.empty:
    # Get the first matched row
    desiredFunc = matchedRow.iloc[0]

    # Execute the matched function
    functionName = desiredFunc['Function title']
    functionToCall = globals()[functionName]  # Get the function by name
    functionToCall(speciesOfInterest)
    print(f"{desiredFunc['Purpose']}")
  else:
    print("Function name not inputed correctly.")

In [ ]:
import pandas as pd

# Define the function titles, steps, and purposes
functionsData = [
    {"Function title": "project", "Part": "Georeferencing", "Purpose": "Projects tif from 1927 to 1983.", "Next Task": "Run createFC function."},
    {"Function title": "createFC", "Part": "Part 1", "Purpose": "Creates feature classes to digitize.", "Next Task": "Start digitizing, then run next."},
    {"Function title": "postDigitize", "Part": "Part 1", "Purpose": "Runs post digitizing steps.", "Next Task": "Remove any internal polygons, then run next."},
    {"Function title": "postEdit", "Part": "Part 1", "Purpose": "Runs the steps after digitization.", "Next Task": "Part 2 work is next."},
    {"Function title": "postEdit2", "Part": "Part 1", "Purpose": "Runs the steps after digitization (points).", "Next Task":"None."},

    {"Function title": "changeUpdate", "Part": "Part 2", "Purpose":"Update file to lines.", "Next Task":"Start digitizing the dashed lines."},
    {"Function title": "dashLinesToPolygon", "Part": "Part 2", "Purpose":"Converts dashed lines to polygons.", "Next Task":"Start selecting, do the copy and paste special."},
    {"Function title": "finalStep", "Part": "Part 2", "Purpose": "Fills in update for level 3 and also export it.", "Next Task":"None."}
]
# Create a DataFrame from the functions data
functionsDF = pd.DataFrame(functionsData)

#### 2. Setting the file paths
These workflowFolderLoc and geodatabase are unique to the user's computer. Edit the filepath accordingly since it is hardcoded. 

In [1]:
# file paths
workflowFolderLoc = r"E:\Thorne_Lab_Internship\G_and_F_workflow1" # edit the location 
geodatabase = f"{workflowFolderLoc}\\g_and_f.gdb" # edit the gdb 

#### 3. Digitizing and running the script
After running the necessary functions and setting the file paths, it's time to input in your species of interest and start digitizing.

In [ ]:
# defining species of interest
speciesOfInterest = input("Input species name of interest: ").lower() # the abbreviation

# defining digitizing
pointResult = input("Does the map have points? (yes/no) ").lower() 
polygonResult = input("Does the map have polygons? (yes/no) ").lower() 
probableResult = input("Does the map have the dashed lines? (yes/no) ").lower() 

In [ ]:
# run for reference in order to see function title, part, purpose, and next task
functionsDF

In [ ]:
# Get user input
userInput = input("Put in your desired step aka the Function title: ")
executeFunction(userInput)

#### 4. Clipping feature classes to boundary, etc. 

End of project things.

In [ ]:
featureDatasets = arcpy.ListDatasets() # lists all feature datasets - all the ones in the OG gdb

In [11]:
# run for the second time

# paths 
gdb1 = "E:\Thorne_Lab_Internship\G_and_F_workflow1\g_and_f.gdb"
gdb2 = "E:\Thorne_Lab_Internship\G_and_F_workflow1\clipped_g_and_f.gdb"

# list of all feature datasets
arcpy.env.workspace = gdb1
feature_classes_gdb1 = set(arcpy.ListDatasets())
arcpy.env.workspace = gdb2
feature_classes_gdb2 = set(arcpy.ListDatasets())

# compare feature classes
only_in_gdb1 = feature_classes_gdb1 - feature_classes_gdb2
only_in_gdb2 = feature_classes_gdb2 - feature_classes_gdb1

featureDatasets = []

print("Feature classes only in GDB1:")
for fc in only_in_gdb1:
    print(fc)
    featureDatasets.append(fc)
    
print("\nFeature classes only in GDB2:")
for fc in only_in_gdb2:
    print(fc)

Feature classes only in GDB1:
cere

Feature classes only in GDB2:


In [13]:
# DON'T RUN THIS AGAIN

fDatasetsPaths = []

arcpy.env.workspace = geodatabase
clippedGDB = f"{workflowFolderLoc}\\clipped_g_and_f.gdb"

desc = arcpy.Describe(f"{geodatabase}\\abam\\abam_update")
sr = desc.spatialReference
print(sr)

# loop through each feature dataset to create a new one in the clipped GDB
for dataset in featureDatasets:
    newFDS = f"{clippedGDB}\\{dataset}"
    
    try:
        # create the feature dataset with the correct parameters
        arcpy.management.CreateFeatureDataset(clippedGDB, dataset, sr)
        print(f"Successfully created feature dataset: {dataset}")
    except Exception as e:
        print(f"Error creating feature dataset {dataset}: {e}")
        
    # for that same feature dataset, I also want to list the feature classes within it 
    fcList = arcpy.ListFeatureClasses(feature_dataset=dataset)
    for fc in fcList:
        print(fc)
        arcpy.gapro.ClipLayer(
        input_layer=fc,
        clip_layer=f"{geodatabase}\\CA_bnd83",
        out_feature_class = f"{clippedGDB}\\{dataset}\\{fc}"
        )

<geoprocessing spatial reference object object at 0x0000024CF2C72950>
Successfully created feature dataset: cere
cere_points


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 003917: The feature class geometry is not of type point | polygon | polyline
Failed to execute (ClipLayer).


In [ ]:
# for later - checking to see what tifs I'm missing

for feature in featureDatasets:
    tif27path = f"{workflowFolderLoc}\\georef_1927\\{feature}"
    tif83path = f"{workflowFolderLoc}\\georef_1983\\{feature}"

    if not arcpy.Exists(tif27path):
        print(f"{tif27path} does not exist.")

    if not arcpy.Exists(tif83path):
        print(f"{tif83path} does not exist.")

In [ ]:
desc = arcpy.Describe(r"E:\Thorne_Lab_Internship\G_and_F_workflow1\g_and_f.gdb\abbr")
sr = desc.spatialReference
print(sr)

# lists all fds's with level 3 

level3ds = []

for dataset in featureDatasets: 
    fcList = arcpy.ListFeatureClasses(feature_dataset=dataset)
    # Check if any feature class contains 'level_3' in its name
    if any('level_3' in fc for fc in fcList):
        level3ds.append(dataset)
print(level3ds)

for dataset in level3ds: 
    arcpy.management.CreateFeatureclass(
            out_path=f"{geodatabase}\\{dataset}",
            out_name=f"{dataset}_dashlines",
            geometry_type="POLYLINE",
            template=None,
            has_m="DISABLED",
            has_z="DISABLED",
            spatial_reference='PROJCS["NAD_1983_2011_California_Teale_Albers",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-16909700 -8597000 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision',
            config_keyword="",
            spatial_grid_1=0,
            spatial_grid_2=0,
            spatial_grid_3=0,
            out_alias="",
            oid_type="SAME_AS_TEMPLATE")     